# 🚗 Car License Plate Recognition System
## Professional Analysis, Implementation, and Usage Guide

### Team Members
- **Ahmed Al-Duais** - 202270176
- **Abulkareem Thiab** - 202270136
- **Ayman Mrwan** - 202270324

---

### Table of Contents
1. Project Overview
2. System Architecture
3. Setup and Model Assets
4. GUI Applications and Usage
5. Data Preparation
6. Model Training
7. Inference and Testing
8. Real-time Detection
9. Results and Performance
10. Future Improvements



## 1) Project Overview

This project implements an end-to-end car license plate detection and recognition system tailored for Yemeni plates. It uses YOLOv8 for plate detection and EasyOCR for Arabic/English text recognition, with multiple GUIs for real-time and batch workflows.


## 2) System Architecture

```mermaid
graph TD
    A[Input Images/Video] --> B[Data Preprocessing]
    B --> C[YOLOv8 Model Training]
    C --> D[Model Validation]
    D --> E[Inference Engine]
    E --> F[Real-time Detection]
    F --> G[Output Results]
    
    H[Live Stream] --> I[Frame Processing]
    I --> E
    
    J[CSV Annotations] --> K[YOLO Format Conversion]
    K --> B
```

Core Modules:
- Data pipeline: `convert_to_yolo.py`
- Training: `main.py`, `yemen car plate number model train.py`
- Inference/Utilities: `model_test.py`, `seperated-plates.py`
- Real-time: `recognition-with-live-stream/live-stream.py`


## 3) Setup and Model Assets

- Python 3.13.5+
- Install dependencies (UV recommended):
```bash
pip install uv
uv sync
```
- Verify:
```bash
python -c "import ultralytics, easyocr, cv2; print('OK')"
```

Models:
- Primary plate detector: `src/models/yolov8s14/weights/best.pt`
- Component detector (text/city/number): `src/models/yolov8s5/weights/best.pt`
- Base models: `yolo11n.pt`, `yolo11s.pt`, `yolov8n.pt`, `yolov8s.pt`


## 4) GUI Applications and Usage

- Advanced Qt GUI (tabs + live stream):
```bash
python GUIs/qt_with_live/main.py
```
- Simple Qt GUI (single image):
```bash
python GUIs/simple-gui-qt.py
```
- Two‑Stage GUI:
```bash
python yemen_plate_gui.py
```
- Enhanced Two‑Stage GUI (latest):
```bash
python yemen_plate_v2_gui.py
```
- Simple Tkinter GUI:
```bash
python GUIs/simple_gui_tiknter/simple_gui_tiknter.py
```


In [ ]:
# Environment and project scan
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette("husl")

print("📁 Project Structure (top-level):")
for path in sorted(Path('.').iterdir()):
    if path.name.startswith('.'):
        continue
    print('-', path)



## 5) Data Preparation

Analyze dataset structure and distribution for both datasets.


In [ ]:
# Dataset analysis utilities
import os
import numpy as np
import matplotlib.pyplot as plt

def analyze_dataset_structure():
    datasets = {
        "Yemen Car Plate v1i": "src/data/plat number car yemen.v1i.yolov8",
        "Yemen Plate": "src/data/yemen-plate"
    }
    results = {}
    for name, path in datasets.items():
        if os.path.exists(path):
            train_images = len([f for f in os.listdir(f"{path}/train/images") if f.lower().endswith(('.jpg','.jpeg','.png'))])
            val_images = len([f for f in os.listdir(f"{path}/valid/images") if f.lower().endswith(('.jpg','.jpeg','.png'))])
            test_images = len([f for f in os.listdir(f"{path}/test/images") if f.lower().endswith(('.jpg','.jpeg','.png'))])
            results[name] = {
                'train': train_images,
                'validation': val_images,
                'test': test_images,
                'total': train_images + val_images + test_images
            }
    return results

def visualize_dataset_stats(stats):
    if not stats:
        print('No dataset stats available')
        return
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    datasets = list(stats.keys())
    train_counts = [stats[ds]['train'] for ds in datasets]
    val_counts = [stats[ds]['validation'] for ds in datasets]
    test_counts = [stats[ds]['test'] for ds in datasets]
    x = np.arange(len(datasets)); width = 0.25
    ax1.bar(x - width, train_counts, width, label='Train', alpha=0.8)
    ax1.bar(x, val_counts, width, label='Validation', alpha=0.8)
    ax1.bar(x + width, test_counts, width, label='Test', alpha=0.8)
    ax1.set_xlabel('Dataset'); ax1.set_ylabel('Number of Images'); ax1.set_title('Dataset Distribution Comparison')
    ax1.set_xticks(x); ax1.set_xticklabels(datasets, rotation=45); ax1.legend(); ax1.grid(True, alpha=0.3)
    primary = "Yemen Car Plate v1i"
    if primary in stats:
        sizes = [stats[primary]['train'], stats[primary]['validation'], stats[primary]['test']]
        labels = ['Train','Validation','Test']
        ax2.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
        ax2.set_title(f'{primary} Split Distribution')
    plt.tight_layout(); plt.show()

stats = analyze_dataset_structure()
visualize_dataset_stats(stats)
print(stats)


## 6) Model Training

Train plate and component models with YOLOv8.


In [ ]:
# Example training snippets (commented for safety)
# from ultralytics import YOLO
# model = YOLO('yolov8s.pt')
# results = model.train(
#     data='src/data/yemen-plate/data.yaml',
#     epochs=50,
#     imgsz=640,
#     batch=9,
#     device=0,
#     name='yolov8s',
#     workers=0,
# )

print('Refer to main.py or yemen car plate number model train.py for full training scripts.')


## 7) Inference and Testing

Run demos: basic detection + OCR, and two-stage detection + enhanced OCR.


In [ ]:
# Basic detection + OCR demo (from Project notebook)
from pathlib import Path
import cv2
from ultralytics import YOLO
import easyocr

primary_model = Path('src/models/yolov8s14/weights/best.pt')
example_dir = Path('src/data/plat number car yemen.v1i.yolov8/test/images')

if primary_model.exists():
    model = YOLO(str(primary_model))
    reader = easyocr.Reader(['en','ar'])
    img_path = None
    if example_dir.exists():
        for p in example_dir.iterdir():
            if p.suffix.lower() in {'.jpg','.jpeg','.png'}:
                img_path = p; break
    if img_path:
        img = cv2.imread(str(img_path))
        results = model(img)
        for r in results:
            if r.boxes is None: continue
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                crop = img[y1:y2, x1:x2]
                if crop.size == 0: continue
                ocr = reader.readtext(crop)
                text = " ".join([t[1] for t in ocr])
                if 'خصوصي' in text or 'نقل' in text:
                    text = 'خصوصي نقل اجرة'
                print('Detected:', text)
    else:
        print('No example images found.')
else:
    print('Model not found:', primary_model)


In [ ]:
# Two-stage detection + enhanced OCR demo (from Project notebook)
from pathlib import Path
import cv2, numpy as np
from ultralytics import YOLO
import easyocr

def arabic_to_english_digits(text: str) -> str:
    return text.translate(str.maketrans("٠١٢٣٤٥٦٧٨٩", "0123456789"))

def preprocess_for_arabic_ocr(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape
    if h < 50 or w < 100:
        sf = max(50/h, 100/w, 2.0)
        gray = cv2.resize(gray, (int(w*sf), int(h*sf)), interpolation=cv2.INTER_CUBIC)
    blurred = cv2.GaussianBlur(gray, (3,3), 0)
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8,8))
    enhanced = clahe.apply(blurred)
    filtered = cv2.bilateralFilter(enhanced, 9, 75, 75)
    kernel = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
    sharpened = cv2.filter2D(filtered, -1, kernel)
    thresh = cv2.adaptiveThreshold(sharpened, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)

def extract_text_multi(image, reader, conf_th=0.2):
    results = []
    variants = [
        ("preprocessed", preprocess_for_arabic_ocr(image)),
        ("original", image),
        ("grayscale", cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR)),
        ("enhanced", cv2.convertScaleAbs(image, alpha=1.5, beta=30)),
        ("inverted", cv2.bitwise_not(image))
    ]
    for name, img in variants:
        try:
            ocr = reader.readtext(img, paragraph=False, width_ths=0.7, height_ths=0.7, detail=1)
            for box, text, conf in ocr:
                if conf >= conf_th:
                    results.append({"text": text, "confidence": conf, "approach": name})
        except Exception:
            pass
    return results

plate_model = Path('src/models/yolov8s14/weights/best.pt')
detail_model = Path('src/models/yolov8s5/weights/best.pt')
example_dir = Path('src/data/plat number car yemen.v1i.yolov8/test/images')

if plate_model.exists() and detail_model.exists():
    plate = YOLO(str(plate_model))
    details = YOLO(str(detail_model))
    reader = easyocr.Reader(['en','ar'], gpu=False)
    img_path = None
    if example_dir.exists():
        for p in example_dir.iterdir():
            if p.suffix.lower() in {'.jpg','.jpeg','.png'}:
                img_path = p; break
    if img_path:
        img = cv2.imread(str(img_path))
        out = []
        for r in plate(img):
            if r.boxes is None: continue
            for b in r.boxes:
                x1,y1,x2,y2 = map(int, b.xyxy[0].cpu().numpy())
                crop = img[y1:y2, x1:x2]
                if crop.size == 0: continue
                city, text_ar, num_ar = '', '', ''
                for dr in details(crop, conf=0.1):
                    if dr.boxes is None: continue
                    for db in dr.boxes:
                        dx1,dy1,dx2,dy2 = map(int, db.xyxy[0].cpu().numpy())
                        cls = int(db.cls[0].cpu().numpy())
                        label = dr.names[cls]
                        sub = crop[dy1:dy2, dx1:dx2]
                        if sub.size == 0: continue
                        ocrs = extract_text_multi(sub, reader)
                        if ocrs:
                            best = max(ocrs, key=lambda x: x['confidence'])
                            t = best['text']
                            if label == 'city':
                                city = t
                            elif label == 'text':
                                if 'خصوصي' in t or 'نقل' in t:
                                    t = 'خصوصي نقل اجرة'
                                text_ar = t
                            elif label == 'number':
                                num_ar = t
                out.append({'city': city, 'text': text_ar, 'number_ar': num_ar, 'number_en': arabic_to_english_digits(num_ar) if num_ar else ''})
        print(out)
    else:
        print('No example images found.')
else:
    print('Required models not found.')


## 8) Real-time Detection

Command-line live stream script:
```bash
python recognition-with-live-stream/live-stream.py
```
Edit `STREAM_URL` inside the script.


## 9) Results and Performance

- Trained YOLOv8s model (50 epochs)
- Refer to `runs/detect/yolov8n14/results.csv` for metrics (mAP, precision, recall)
- Confusion matrices and curves available in `runs/detect/yolov8n14/`


## 10) Future Improvements

- Expand datasets and annotations for robustness
- Improve Arabic OCR post-processing and language models
- Export and optimize models for deployment (ONNX/TensorRT)
- Add unit tests and CI for training/inference pipelines
